# 連立常微分方程式
連立常微分方程式を解く場合も1次元の場合と同じ．  
## 例：ローレンツ方程式  
<br>
<!-- <img src="https://latex.codecogs.com/png.image?\dpi{120}&space;\bg_white&space;\begin{pmatrix}\dot{x}\\&space;\dot{y}\\&space;\dot{z}\end{pmatrix}=\begin{pmatrix}-px&plus;py\\-xz&plus;rx-y\\&space;xy-bz\\\end{pmatrix}" title="\bg_white \begin{pmatrix}\dot{x}\\ \dot{y}\\ \dot{z}\end{pmatrix}=\begin{pmatrix}-px+py\\-xz+rx-y\\ xy-bz\\\end{pmatrix}" />  
<br> -->

$$
\begin{bmatrix}
\dot{x}\\
\dot{y}\\
\dot{z}\\
\end{bmatrix} = \begin{bmatrix}
\ -px+py\\
\ -xz+rx-y\\
\ xy-bz\\
\end{bmatrix}
$$

ただし  

$$
\begin{bmatrix}
\ x_0\\
\ y_0\\
\ z_0\\
\end{bmatrix} = \begin{bmatrix}
\ 0\\
\ 4\\
\ 28\\
\end{bmatrix},  \quad  p = 10,  \quad  r = 28,  \quad  b = \frac{8}{3}
$$

とする．  

In [1]:
using DifferentialEquations
using ParameterizedFunctions

ここでは`@ode_def`マクロを使う  

In [2]:
### 微分方程式の定義 ###
lorenz_m = @ode_def begin
    dx = -p*x + p*y
    dy = -x*z + r*x -y
    dz = x*y - b*z
end p r b


### 問題を定義 ###
const x₀ = [0.0, 4.0, 28.0]
const p = 10
const r = 28
const b = 8/3
const t_span = (0, 50.0)
prob = ODEProblem(lorenz_m, x₀, t_span, (p, r, b))


### 解く ###
sol = solve(prob);

### (微分方程式の描画)
`Latexify`を使うと定義した微分方程式を綺麗に表示できる．  
微分方程式のチェック？などに使えるかも  

In [64]:
using Latexify
latexify(lorenz_m)

L"\begin{align}
\frac{dx(t)}{dt} =& p y\left( t \right) - p x\left( t \right) \\
\frac{dy(t)}{dt} =&  - y\left( t \right) + r x\left( t \right) - x\left( t \right) z\left( t \right) \\
\frac{dz(t)}{dt} =& x\left( t \right) y\left( t \right) - b z\left( t \right)
\end{align}
"

### 解のプロット

In [68]:
using Plots

plot(sol)
savefig("sb2_1.png")

![pic](picture/sb2_1.png)

### 解について
解は`ODECompositeSolution`として返される．  
`sol[i]`でi番目の解にアクセスできる．  

In [69]:
sol[2]

3-element Vector{Float64}:
  0.0014137366927165365
  3.999858601385078
 27.997360736072547

`sol.t` : 時刻歴の配列

In [70]:
sol.t[1:5]

5-element Vector{Float64}:
 0.0
 3.535028953698351e-5
 0.0003888531849068186
 0.003923882138605169
 0.020333768098013315

`sol.u` : 解ベクトルの配列

In [71]:
sol.u[1:5]

5-element Vector{Vector{Float64}}:
 [0.0, 4.0, 28.0]
 [0.0014137366927165365, 3.999858601385078, 27.997360736072547]
 [0.015520905424812406, 3.998444948030281, 27.970992749143974]
 [0.15361282439337523, 3.98439393599714, 27.709751122448132]
 [0.7286763765799963, 3.9268855053350147, 26.551972589469933]

`sol.prob` : 初期値問題の設定内容

In [72]:
sol.prob

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 50.0)
u0: 3-element Vector{Float64}:
  0.0
  4.0
 28.0

他にも色々なデータが入ってます．  
おもしろい機能として解の補完があります．  
```julia
sol(時間)
```
で`t=時間`での補完した解が得られます．  
`[]`ではなく`()`なことに注意  

In [73]:
sol(1.2345)

3-element Vector{Float64}:
 11.736388307019535
  6.081725428276198
 36.57200368170504

この機能があるため`scipy.integrate.solve_ivp`と異なり，刻み時間を指定しなくても初めからプロットがなめらかです．  
せっかくなので解を3次元でプロットしてみます．

In [74]:
plot(sol, vars=(1,2,3))
savefig("sb2_2.png")

![pic](picture/sb2_2.png)

****
## 高速化
微分方程式の定義方法によって計算速度はかなり変動します．  
公式wiki : [https://diffeq.sciml.ai/stable/tutorials/faster_ode_example/#Code-Optimization-in-Julia](https://diffeq.sciml.ai/stable/tutorials/faster_ode_example/#Code-Optimization-in-Julia)

In [75]:
"""ローレンツ方程式"""
function lorenz(X, param, t)
    x, y, z = X
    p, r, b = param

    dx = -p*x + p*y
    dy = -x*z + r*x -y
    dz = x*y - b*z
    
    [dx, dy, dz]  # ベクトルを返す
end

prob2 = ODEProblem(lorenz, x₀, t_span, (p, r, b))

ODEProblem with uType Vector{Float64} and tType Float64. In-place: false
timespan: (0.0, 50.0)
u0: 3-element Vector{Float64}:
  0.0
  4.0
 28.0

上のように速度をベクトルで返しても解けますが，配列に書き出すほうがかなり高速です  

In [76]:
"""ローレンツ方程式．配列に書き出す方式

juliaでは引数を破壊するような関数には!をつける
"""
function lorenz!(dX, X, param, t)
    x, y, z = X
    p, r, b = param

    dX[1] = -p*x + p*y
    dX[2] = -x*z + r*x -y
    dX[3] = x*y - b*z
    
    nothing  # 何も返さない
end

prob3 = ODEProblem(lorenz!, x₀, t_span, (p, r, b))

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 50.0)
u0: 3-element Vector{Float64}:
  0.0
  4.0
 28.0

### ベンチマーク
BenchmarkTools.jlを使って比較

In [77]:
using BenchmarkTools

1. 配列を返す：`lonrenz`

In [81]:
bench1 = @benchmark solve(prob2)  # 書き出さない

BenchmarkTools.Trial: 2416 samples with 1 evaluation.
 Range (min … max):  1.020 ms … 11.365 ms  ┊ GC (min … max):  0.00% … 81.75%
 Time  (median):     2.157 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.064 ms ±  1.778 ms  ┊ GC (mean ± σ):  18.26% ± 17.34%

  █▂ ▁▁  ▇▆▂                                           ▁     ▁
  ██████████▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▇▇██▇▆▃ █
  1.02 ms      Histogram: log(frequency) by time     9.67 ms <

 Memory estimate: 6.06 MiB, allocs estimate: 56053.

2. 配列書き出し：`lorenz!`

In [79]:
bench2 = @benchmark solve(prob3)  # 書き出し

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  194.759 μs …   8.940 ms  ┊ GC (min … max):  0.00% … 89.06%
 Time  (median):     214.483 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   311.209 μs ± 588.332 μs  ┊ GC (mean ± σ):  14.06% ±  7.21%

  ▁█▅                                 ▂▃▁                        
  ████▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▅███▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  195 μs           Histogram: frequency by time          421 μs <

 Memory estimate: 747.84 KiB, allocs estimate: 6185.

2.の`lorenz!`のほうがメモリ使用量も少なく，かなり高速なことがわかる．  
<br>
`@ode_def`マクロを使って微分方程式を定義すると，自動的に配列書き出しの関数定義になる．  

3. @ode_def

In [80]:
bench3 = @benchmark solve(prob)  # @ode_def

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  195.039 μs …   9.081 ms  ┊ GC (min … max):  0.00% … 88.98%
 Time  (median):     216.262 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   311.521 μs ± 588.314 μs  ┊ GC (mean ± σ):  13.89% ±  7.13%

  ▁█▄                              ▂▃                            
  ███▇▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▅███▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂ ▃
  195 μs           Histogram: frequency by time          443 μs <

 Memory estimate: 736.95 KiB, allocs estimate: 6074.

2.配列書き出しとほぼ同じメモリ使用量，計算時間なことがわかる